In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
pd.options.display.max_columns = 50
pd.options.display.max_rows = 100
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score
from operator import itemgetter
import seaborn as sns

In [2]:
def add_data_info():
    df1 = pd.read_csv('https://raw.githubusercontent.com/JeffSackmann/tennis_MatchChartingProject/master/charting-m-points-from-2017.csv', encoding= 'unicode_escape', low_memory=False)
    df2 = pd.read_csv('raw_data/charting-m-matches.csv', encoding= 'unicode_escape', low_memory=False)
    
    #get separated values from m-matches list of games - based on matchid
    df = pd.merge(df1,df2[['match_id','Date','Tournament','Round','Surface','Player 1','Player 2']], on=['match_id'], how='left')
    
    #add the condition of "Grand Slam" - 5 or 3 sets
    conditions = df['Tournament'].isin(["Wimbledon", "Australian Open", "US Open", "Roland Garros"])
    values = [1,0]

    # Create a new column  based on the conditions and values
    df['Grand Slam'] = pd.Series(pd.NA)
    df.loc[conditions, 'Grand Slam'] = values[0]
    df.loc[~conditions, 'Grand Slam'] = values[1]
    
    # save the original dataset with enriched data for future features
    df.to_csv('raw_data/charting-m-points-from-2017-enriched.csv', index=False)

In [3]:
def filter_player(df,opponent):
    
    # Read the dataset into a pandas DataFrame - from enriched file
    df = pd.read_csv(df, encoding= 'unicode_escape', low_memory=False)
    
    ## filtra pelo  oponente
    df = df[(df["Player 1"].str.contains(opponent)) | (df["Player 2"].str.contains(opponent))].copy()
    
    ##define se está sacando e se ganhou o ponto
    df['i_serve'] = np.where(((df['Svr'] == 1) & (df['Player 1'] == opponent )) | ((df['Svr'] == 2) & (df['Player 2'] == opponent)), 0, 1)
    df['i_win'] = np.where(((df['isSvrWinner'] == df['i_serve'])), 1, 0)
    df['is_second_service'] = np.where(((df['2nd'].isnull())), 0, 1)
    
    ## direção do saque
    # primeiro caractere do segundo saque
    df['dir_srv'] = df.loc[:, '2nd'].str[0]
    ## completa com o primeiro caractere do primeiro saque quando ele entrou
    df['dir_srv'].fillna(df.loc[:, '1st'].str[0], inplace=True)

    
    ##reseta o indice
    df.reset_index(inplace=True)
    
    ## salva arquivo e retorna dataset da funcao
    csv_file = "raw_data/" + opponent + "_points.csv"
    df.to_csv(csv_file, index=False)
    return df

In [4]:
meaning_char = pd.read_csv('https://raw.githubusercontent.com/eduardoscovino/tennis-analytics/master/code_map.csv',
                           encoding= 'unicode_escape', sep=';')

In [ ]:
add_data_info()

In [ ]:
df = filter_player('raw_data/charting-m-points-from-2017-enriched.csv','Novak Djokovic')

In [ ]:
## serve direction encode 
df = df[df['dir_srv'].isin(['4','5','6'])]

## to numeric for rallyCount and serve direction
df['rallyCount']=pd.to_numeric(df['rallyCount'])
df['dir_srv']=pd.to_numeric(df['dir_srv'])

In [ ]:
#Spliting Serve and Return
df['rally_desc'] = df.loc[:, '2nd']
df['rally_desc'].fillna(df.loc[:, '1st'], inplace=True)

##df['Serve'] = df.loc[:, 'rally_desc'].str[0:1]

df['rally_desc'] = df['rally_desc'].str[1:]

sc = np.array(meaning_char[~meaning_char['code'].isin(['f', 'b', 'r', 's', 'v', 'l', 'o', 'z', 'p', 'u', 'y', 'h', 'i', 'j', 'k','m', '1', '2', '3', '7', '8', '9'])]['code'])

#Spliting every shot
df['rally_desc'] = df['rally_desc'].apply(lambda x: ''.join([c for c in x if c not in sc]))
df['rally_desc'] = df['rally_desc'].str.replace('f',' f').str.replace('b',' b').str.replace('s',' s').str.replace('r',' r').str.replace('v',' v').str.replace('l',' l').str.replace('o',' o').str.replace('z',' z').str.replace('p',' p').str.replace('u',' u').str.replace('y',' y').str.replace('h',' h').str.replace('i',' i').str.replace('j',' j').str.replace('k',' k').str.replace('m',' m').str.replace('!','').str.replace('+','').str.replace(';','').str.replace('^','').str.replace('C','')
df['rally_desc'] = df['rally_desc'].map(lambda x: x.lstrip(' ').rstrip(' '))

In [ ]:
df['rally_tratada'] = (df['rallyCount']/2).apply(np.floor)

In [ ]:
df.head(20)

### Adição da coluna how_ended

In [ ]:
# df['end']
df['how_ended'] = df.loc[:, '2nd'].str[-1]
df['how_ended'].fillna(df.loc[:, '1st'].str[-1], inplace=True)

df.head()

In [ ]:
# vamos entender melhor isso aqui

df['how_ended'].value_counts()

In [ ]:
df['isUnforced'] = df['isUnforced'].replace({True: 1, False: 0})
df['isForced'] = df['isForced'].replace({True: 1, False: 0})
df['serve_return'] = df['rally_desc'].map(lambda x: x.split(' ')).str[0]

## Dataset finalizado ##

In [ ]:
df['serve_return'].nunique()

In [ ]:
df['how_ended'].nunique()

In [ ]:
#df = df[df['serve_return']!=""]

In [ ]:
serve_return_column = df["serve_return"]
value_counts = serve_return_column.value_counts()

plt.bar(value_counts.index, value_counts.values )
plt.xlabel("serve_return")
plt.ylabel("Frequency")
plt.title("Distribution of serve_return")
plt.xticks(rotation=90)
plt.show()

In [ ]:
serve_return_counts = df["serve_return"].value_counts().reset_index()
serve_return_counts.columns = ["serve_return", "frequency"]
serve_return_counts = serve_return_counts.sort_values("frequency", ascending=False )

serve_return_counts.head(10)


In [ ]:
rally_count_dist = df["rallyCount"]
rally_count_dist = rally_count_dist.dropna()

plt.hist(rally_count_dist)
plt.xlabel("rally_count_dist")
plt.ylabel("Frequency")
plt.title("Distribution of rally count")
plt.show()

In [ ]:
rally_counts = df["rallyCount"].value_counts().reset_index()
rally_counts.columns = ["rally_counts", "frequency"]
rally_counts = rally_counts.sort_values("frequency", ascending=False )

print(rally_counts)

## Geração das features

In [ ]:
features = ['Surface',
            'Round',
            'Grand Slam',
            'i_serve',
            'is_second_service',
            'dir_srv',
            'rally_tratada',
            # 'rallyCount',
            'isUnforced',
            'isForced',
            "serve_return",
            "how_ended"
            ]
X = df[features]
y = df['i_win']

In [ ]:
feat_categorical = ['Surface','Round','dir_srv','serve_return','how_ended']

In [ ]:
X.info()

## separação - treino, teste e validação

In [ ]:
from sklearn.model_selection import train_test_split

# Train-Test Split - ## parametro stratify ###
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.3, random_state=1)

In [ ]:
print(len(X_train), 'train examples')
print(len(X_test), 'test examples')

In [ ]:
X_train['serve_return'].nunique()

## Inicio do modelo - pre processamento e encoder

In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn import set_config
set_config(display="diagram")

In [ ]:
preproc = make_column_transformer(
    (StandardScaler(), make_column_selector(dtype_exclude=["object"])),
    (OneHotEncoder(handle_unknown="ignore", sparse_output=False), make_column_selector(dtype_include=["object"]))
)

preproc

In [ ]:
preproc.fit_transform(X_train).shape

In [ ]:
X_train_preproc = preproc.fit_transform(X_train)
X_test_preproc = preproc.transform(X_test)

## Definição do modelo

In [ ]:
from tensorflow.keras import Sequential, layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
gb = GradientBoostingClassifier(random_state=5)

params = {"learning_rate": [0.01, 0.1, 0.5],
            "min_samples_split": [5, 10],
            "min_samples_leaf": [3, 5],
            "max_depth":[3,5,10],
            "max_features":["sqrt"],
            "n_estimators":[100, 200]} 

gb_cv = GridSearchCV(gb, params, cv = 3, n_jobs = -1, verbose = False)

gb_cv.fit(X_train_preproc, np.ravel(y_train))

In [ ]:
gb = gb_cv.best_estimator_

In [ ]:
gb

## Resultado final / evaluate

In [ ]:
def plot_auc(model):
    fpr, tpr, _ = roc_curve(y_test, model.predict_proba(X_test_preproc)[:,1]) #test AUC
    plt.figure(figsize=(5,4))
    plt.plot([0, 1], [0, 1], 'k--')
    plt.plot(fpr, tpr, label="test")

    fpr_train, tpr_train, _ = roc_curve(y_train, model.predict_proba(X_train_preproc)[:,1]) #train AUC
    plt.plot(fpr_train, tpr_train, label="train")
    auc_test = roc_auc_score(y_test, model.predict_proba(X_test_preproc)[:,1])
    auc_train = roc_auc_score(y_train, model.predict_proba(X_train_preproc)[:,1])
    plt.legend()
    plt.title('Curva ROC')
    print('AUC score é %.2f em dados de teste e %.2f em dados de treinamento.'%(auc_test, auc_train))
    plt.show()
    
def plot_conf_matrix(model):
    fig, ax = plt.subplots(figsize=(5,4))
    cm = confusion_matrix(y_test, model.predict(X_test_preproc))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")

    ax.xaxis.set_ticklabels(['Derrota', 'Vitória'])
    ax.yaxis.set_ticklabels(['Derrota', 'Vitória']);
    ax.set_xlabel('Labels reais')
    ax.set_ylabel('Labels preditas')
    ax.set_title('Matriz de confusão')
    plt.show()

**O que é curva ROC?**

    ROC é uma curva de probabilidade e AUC representa o quão bem o modelo conseguiu separar as 2 classes, no nosso caso 'Vitória' e 'Derrota'. Quanto mais perto do canto superior esquerdo passa a cuva ROC no gráfico, melhor o desempenho dele, e, quanto maior o valor AUC, melhor o modelo é em prever corretamente essas classes.

In [ ]:
plot_auc(gb_cv.best_estimator_)

**O que é matriz de confusão?**
    
    É uma visualização usada para analisar o desempenho de um classificador, contando as vezes em que o rótulo real e o rótulo previsto são iguais.
    Nesse sentido, quanto maiores os valores na diagonal melhor, pois representa os casos que ambos rótulos reais e preditos foram definidos como "Vitória" ou ambos foram definidos como "Derrota".

In [ ]:
plot_conf_matrix(gb)

In [ ]:
gb_cv.best_score_

In [ ]:
gb_final = gb_cv.best_estimator_

In [ ]:
gb2 = gb.fit(X_train_preproc, y_train)

In [ ]:
gb2.score(X_test_preproc, y_test)

In [ ]:
predictions = gb2.predict(X_test_preproc)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
classification_report(y_test, predictions, output_dict=True)

https://www.statology.org/sklearn-classification-report/

referencia : aula House Kaggle

https://kitt.lewagon.com/camps/1166/challenges?path=05-ML%2F07-Ensemble-Methods%2F01-Houses-Kaggle-Competition